# Friedel Pair Mapping notebook

The idea behind this is similar to the idea in DCT and here https://doi.org/10.1107/S1600576724009634, where Friedel pairs are used to locate where diffraction spots come from in space. In those cases we use peaks that are 180 degrees apart. This notebook is looking for peaks that are separated by twotheta. These are the peaks we use in the friedel_rois macro at ID11 that aligns grains on the centre of rotation.

The pairs we use will have:
- eta -> -eta
- tth -> tth
- gve -> -gve

Jon Wright. March 2025.


In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# python environment stuff
IMAGED11_PATH = None  # means do not use git, otherwise "ImageD11" or "ImageD11_version_xx", etc
CHECKOUT_PATH = None  # None means guess, or you can specify a folder for the checkout

dset_path = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

y0_guess = 0.0
gvtol = 0.002     # value is often OK

In [ ]:
if IMAGED11_PATH is not None:
    exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
    setup_ImageD11_from_git(CHECKOUT_PATH, IMAGED11_PATH)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial
import ImageD11.sinograms.dataset
from tqdm.autonotebook import tqdm
from scipy.optimize import curve_fit

%matplotlib ipympl

In [ ]:
ds = ImageD11.sinograms.dataset.load(dset_path)
print(ds)

In [ ]:
cf_4d = ds.get_cf_4d()
ds.update_colfile_pars(cf_4d)
print(cf_4d.nrows/1e6, "million peaks read in")

In [ ]:
def find_pairs_minus_eta( cf, gvtol = 0.002 ):
    """
    Locate Friedel pairs with eta -> -eta and g -> -g
    returns ip, im  == indices for eta+ and eta- pairs
    """
    # select peaks from left or right of detector
    fp = np.mgrid[0:cf.nrows][cf_4d.eta > 0 ]
    fm = np.mgrid[0:cf.nrows][cf_4d.eta < 0 ]
    # gvector arrays of these peaks,  make into KD trees
    kdp = scipy.spatial.cKDTree(  np.transpose( (cf.gx[fp], cf.gy[fp], cf.gz[fp]) ) )
    kdm = scipy.spatial.cKDTree( -np.transpose( (cf.gx[fm], cf.gy[fm], cf.gz[fm]) ) )
    # Find the pairs
    coo = kdp.sparse_distance_matrix( kdm, gvtol, output_type = 'coo_matrix' )
    # Return the indices in the original cf_4d
    return fp[coo.row], fm[coo.col]

def locate_pairs( cf, pairs, y0 = 0. ):
    """
    Fit the centre of mass position of the pairs
    cf = colfile
    pairs = (ip, im) = indices of low, high pair in cf
    
    Returns sx, sy == sample x and y co-ordinates of the peak-pair
    """
    ip, im = pairs
    r = np.radians(cf.omega )
    so = np.sin(r)
    co = np.cos(r)
    # For each paired peak take dty - y0 == observed dty value
    y = np.transpose((cf.dty[ip]-y0, cf.dty[im]-y0 ))
    # Find the 2x2 matrix for fitting the dty position (-,-) in geometry notebook
    R = np.transpose( (( -so[ip], -co[ip] ),
                       ( -so[im], -co[im] )), axes=(2,0,1))
    # Solve for x,y in the sample co-ordinates
    return np.linalg.solve( R, y ).T

The next cell is locating the Friedel pairs. It seems to need about 1 second per million peaks.

In [ ]:
ip, im = find_pairs_minus_eta( cf_4d, gvtol=gvtol )
print('Got',len(ip),'pairs from',cf_4d.nrows,'peaks, fraction paired =',len(ip)*2/cf_4d.nrows )

Now fit the positions with the y0 guess. Should be faster than finding the pairs.

In [ ]:
sx, sy = locate_pairs( cf_4d, (ip,im), y0 = y0_guess )
hist_guess = np.histogram2d(sx, sy, bins=ds.ybinedges)[0]

In [ ]:
fig, ax = plt.subplots(layout='constrained', figsize=(8,8))
ax.pcolormesh(ds.ybinedges, ds.ybinedges, hist_guess)
ax.set_aspect(1)
ax.set(title=f'y0 guess: {y0_guess}', xlabel='Sample Y axis -->', ylabel='Sample X axis -->')
plt.show()

In [ ]:
# now we try this with a range of y0 guesses, and look for peaks in the standard deviation

In [ ]:
# guess += 5 from y0_guess, you can change this as needed
y0_min = y0_guess - 5
y0_max = y0_guess + 5
n_y0 = 200
y0s = np.linspace(y0_min, y0_max, n_y0)

In [ ]:
hists = np.array([np.histogram2d(*(locate_pairs( cf_4d, (ip,im), y0=y0)), bins=ds.ybinedges)[0] for y0 in tqdm(y0s)])

In [ ]:
stdevs = np.std(hists, axis=(1, 2))
maxs = np.max(hists, axis=(1, 2))

In [ ]:
fig, ax = plt.subplots()
ax.plot(y0s, stdevs, label='stdev')
ax2 = ax.twinx()
ax2.plot(y0s, maxs, color='r', label='max')
fig.legend()
ax.set(xlabel='y0', ylabel='stdev')
ax2.set(ylabel='max')
plt.show()

In [ ]:
# def Gauss(x, C, A, mu, sigma):
#     return C + A*np.exp(-(x-mu)**2/(2.*sigma**2))

# def Cauchy(x, x0, gamma, height, amp):
#     return height + amp * (1/(np.pi * gamma * (1 + ((x-x0)/gamma)**2)))

def Laplace(x, b, mu, height, amp):
    return height + (amp/(2*b)) * np.exp(-(np.abs(x-mu)/b))

In [ ]:
# Cauchy fit

# p0_C = [y0s[np.argmax(stdevs)], 1, np.max(stdevs), 1]
# print(p0_C)
# bounds_C = ([-np.inf, 0, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf])
# parameters_C, _ = curve_fit(Cauchy, y0s, stdevs, method='trf', bounds=bounds_C, p0=p0_C)
# fit_y_cauchy = Cauchy(y0s, *parameters_C)
# print(parameters_C)
# y0_cauchy = parameters_C[0]
# print(y0_cauchy)

In [ ]:
# laplace fit

p0_L = [1, y0s[np.argmax(stdevs)], np.max(stdevs), 1]
print(p0_L)
bounds_L= ([-np.inf, -np.inf, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf])
parameters_L, _ = curve_fit(Laplace, y0s, stdevs, method='trf', bounds=bounds_L, p0=p0_L)
fit_y_laplace = Laplace(y0s, *parameters_L)
print(parameters_L)
y0_laplace = parameters_L[1]
print(y0_laplace)

In [ ]:
fig, ax = plt.subplots()
ax.plot(y0s, stdevs, label='stdevs')
# ax.plot(y0s, fit_y_cauchy, label='Cauchy fit')
ax.plot(y0s, fit_y_laplace, label='Laplace fit')
ax.axvline(y0_laplace, color='red')
ax.legend()
ax.set(xlabel='y0', ylabel='stdev', title='Fits of stdev')
plt.show()

In [ ]:
# take the results of the Laplace fit:
y0_final = y0_laplace
# just take the max value:
# y0_final = y0s[np.argmax(stdevs)]
# or manually override from your interpretation of the plot:
# y0_final = 0
print(y0_final)

In [ ]:
sx, sy = locate_pairs( cf_4d, (ip,im), y0 = y0_final )
hist_final = np.histogram2d(sx, sy, bins=ds.ybinedges)[0]

In [ ]:
fig, ax = plt.subplots(layout='constrained', figsize=(8,8))
ax.pcolormesh(ds.ybinedges, ds.ybinedges, hist_final)
ax.set_aspect(1)
ax.set(title=f'y0 final: {y0_final}', xlabel='Sample Y axis -->', ylabel='Sample X axis -->')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, layout='constrained', figsize=(12,6), sharex=True, sharey=True)
axs[0].pcolormesh(ds.ybinedges, ds.ybinedges, hist_guess)
axs[0].set_aspect(1)
axs[0].set(title=f'y0 guess: {y0_guess}')
axs[1].pcolormesh(ds.ybinedges, ds.ybinedges, hist_final)
axs[1].set_aspect(1)
axs[1].set(title=f'y0 final: {y0_final}')
fig.supxlabel('Sample Y axis -->')
fig.supylabel('Sample X axis -->')
plt.show()